# 🚦 YOLO Detection Server V22 - Fix YOLOv5 Inference
### FIX: YOLOv5 torch.hub uses model.conf instead of conf= param
### Features:
- ✅ Correct YOLOv5 inference call
- ✅ Detailed debug logs
- ✅ LP_detector.pt + LP_ocr.pt

In [ ]:
# Cell 1: CONFIG
BACKEND_URL = 'https://candidates-advertising-morning-interaction.trycloudflare.com'
NMS_URL = 'https://firewall-video-latinas-yeah.trycloudflare.com'

BACKEND_HTTP_URL = BACKEND_URL
BACKEND_WS_URL = BACKEND_URL.replace('https://', 'wss://').replace('http://', 'ws://') + '/ws/kaggle'

TRACK_LINE_Y = 50
CONFIDENCE = 0.5
FRAME_QUEUE_SIZE = 3
RESULT_QUEUE_SIZE = 10

VEHICLE_CLASSES = ['car', 'truck', 'bus', 'motorcycle', 'bicycle']
camera_keys = {}

BASE_DIR = '/kaggle/input/phat-trien-iot-nang-cao/pytorch/default/2/'
LP_DETECTOR_PATH = BASE_DIR + 'LP_detector.pt'
LP_OCR_PATH = BASE_DIR + 'LP_ocr.pt'
TL_MODEL_PATH = BASE_DIR + 'mhiot-dentinhieu-best-new.pt'

print(f"📡 Backend: {BACKEND_URL}")
print(f"📁 LP Detector: {LP_DETECTOR_PATH}")
print(f"📁 LP OCR: {LP_OCR_PATH}")

In [ ]:
# Cell 2: Install
!pip uninstall -y numpy pillow ultralytics > /dev/null 2>&1
!pip install "numpy<2.0.0" "pillow>=10.3.0" scipy ultralytics opencv-python-headless requests websocket-client supervision --upgrade --quiet
!wget -nc -q https://github.com/ultralytics/assets/releases/download/v8.3.0/yolo11m.pt

import numpy, PIL, supervision
print(f'✅ Numpy {numpy.__version__}, PIL {PIL.__version__}, Supervision {supervision.__version__}')

In [ ]:
# Cell 3: Load Models
import warnings
warnings.filterwarnings('ignore')
import os, torch
from ultralytics import YOLO

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'🚀 Device: {device}')

vehicle_model = None
traffic_light_model = None
lp_detector = None       # YOLOv5 model
lp_detector_type = None  # 'ultralytics' or 'yolov5'
lp_ocr = None
lp_ocr_type = None

def load_model(path, name):
    """Load model and return (model, type)"""
    if not os.path.exists(path):
        print(f'❌ {name}: File not found')
        return None, None
    
    # Try Ultralytics first
    try:
        model = YOLO(path).to(device)
        print(f'✅ {name}: Ultralytics YOLO')
        return model, 'ultralytics'
    except Exception as e:
        print(f'⚠️ {name}: Ultralytics failed, trying YOLOv5... ({str(e)[:50]})')
    
    # Try YOLOv5
    try:
        model = torch.hub.load('ultralytics/yolov5', 'custom', path=path, force_reload=False)
        model = model.to(device)
        model.conf = 0.4  # Set default confidence
        model.iou = 0.45  # Set default IOU
        print(f'✅ {name}: YOLOv5 (conf={model.conf})')
        return model, 'yolov5'
    except Exception as e:
        print(f'❌ {name}: All methods failed: {e}')
    
    return None, None

# Load Vehicle
try:
    vehicle_model = YOLO('yolo11m.pt').to(device)
    print('✅ Vehicle: yolo11m.pt')
except Exception as e:
    print(f'❌ Vehicle: {e}')

# Load TL
traffic_light_model, _ = load_model(TL_MODEL_PATH, 'Traffic Light')

# Load LP Detector (YOLOv5)
lp_detector, lp_detector_type = load_model(LP_DETECTOR_PATH, 'LP Detector')

# Load LP OCR
lp_ocr, lp_ocr_type = load_model(LP_OCR_PATH, 'LP OCR')

print('\n=== MODEL STATUS ===')
print(f'vehicle_model: {"✅" if vehicle_model else "❌"}')
print(f'traffic_light_model: {"✅" if traffic_light_model else "❌"}')
print(f'lp_detector: {"✅" if lp_detector else "❌"} (type: {lp_detector_type})')
print(f'lp_ocr: {"✅" if lp_ocr else "❌"} (type: {lp_ocr_type})')

In [ ]:
# Cell 4: Functions with Debug
import cv2, numpy as np, time, requests, threading, queue, json
from datetime import datetime
from collections import deque
import supervision as sv

camera_trackers = {}
camera_positions = {}
camera_configs = {}
current_traffic_light = {}

# Debug counters
debug_stats = {'lp_attempts': 0, 'lp_detected': 0, 'ocr_attempts': 0, 'ocr_success': 0}

def log(msg):
    print(f'[{datetime.now().strftime("%H:%M:%S")}] {msg}')

def run_yolov5_inference(model, img, conf_thres=0.4):
    """Run YOLOv5 inference correctly"""
    model.conf = conf_thres
    results = model(img)
    return results

def run_ultralytics_inference(model, img, conf=0.4, verbose=False):
    """Run Ultralytics inference"""
    return model(img, conf=conf, verbose=verbose)

def read_plate_ocr(plate_crop):
    """OCR using LP_ocr.pt"""
    global debug_stats
    debug_stats['ocr_attempts'] += 1
    
    if lp_ocr is None:
        log('⚠️ OCR: lp_ocr is None')
        return 'unknown'
    
    try:
        h, w = plate_crop.shape[:2]
        log(f'📝 OCR: Input size {w}x{h}')
        
        if h < 10 or w < 20:
            log('⚠️ OCR: Plate too small')
            return 'unknown'
        
        # Run OCR based on model type
        if lp_ocr_type == 'yolov5':
            log('📝 OCR: Using YOLOv5 inference')
            results = run_yolov5_inference(lp_ocr, plate_crop, conf_thres=0.25)
            
            # YOLOv5 returns pandas dataframe
            if hasattr(results, 'pandas'):
                df = results.pandas().xyxy[0]
                log(f'📝 OCR: Found {len(df)} chars')
                if len(df) >= 4:
                    df = df.sort_values('xmin')
                    text = ''.join(df['name'].astype(str).tolist()).upper().replace(' ', '').replace('-', '')
                    log(f'✅ OCR Result: {text}')
                    debug_stats['ocr_success'] += 1
                    return text
                else:
                    log(f'⚠️ OCR: Only {len(df)} chars (need 4+)')
        else:
            log('📝 OCR: Using Ultralytics inference')
            results = run_ultralytics_inference(lp_ocr, plate_crop, conf=0.25)
            
            if len(results) > 0 and hasattr(results[0], 'boxes'):
                chars = []
                for b in results[0].boxes:
                    x1 = float(b.xyxy[0][0])
                    cls_id = int(b.cls[0])
                    char = results[0].names[cls_id]
                    chars.append((x1, char))
                
                log(f'📝 OCR: Found {len(chars)} chars')
                if len(chars) >= 4:
                    chars.sort(key=lambda x: x[0])
                    text = ''.join([c[1] for c in chars]).upper().replace(' ', '').replace('-', '')
                    log(f'✅ OCR Result: {text}')
                    debug_stats['ocr_success'] += 1
                    return text
                else:
                    log(f'⚠️ OCR: Only {len(chars)} chars (need 4+)')
        
        return 'unknown'
    except Exception as e:
        log(f'❌ OCR Error: {e}')
        import traceback; traceback.print_exc()
        return 'unknown'

def detect_license_plate(frame, x1, y1, x2, y2):
    """Detect plate and run OCR"""
    global debug_stats
    debug_stats['lp_attempts'] += 1
    
    if lp_detector is None:
        if debug_stats['lp_attempts'] <= 3:
            log('⚠️ LP: lp_detector is None')
        return None
    
    try:
        crop = frame[y1:y2, x1:x2]
        crop_h, crop_w = crop.shape[:2]
        
        if debug_stats['lp_attempts'] % 50 == 1:
            log(f'🔍 LP: Processing crop {crop_w}x{crop_h}')
        
        # Run detection based on model type
        if lp_detector_type == 'yolov5':
            results = run_yolov5_inference(lp_detector, crop, conf_thres=0.4)
            
            # YOLOv5 pandas result
            if hasattr(results, 'pandas'):
                df = results.pandas().xyxy[0]
                if len(df) == 0:
                    return None
                
                log(f'🔍 LP: Found {len(df)} plate boxes')
                debug_stats['lp_detected'] += 1
                
                # Get best detection
                best = df.iloc[df['confidence'].idxmax()]
                px1, py1, px2, py2 = int(best['xmin']), int(best['ymin']), int(best['xmax']), int(best['ymax'])
                plate_crop = crop[py1:py2, px1:px2]
                
                log(f'🔍 LP: Plate box [{px1},{py1},{px2},{py2}]')
                
                text = read_plate_ocr(plate_crop)
                if text != 'unknown':
                    return {'text': text, 'confidence': float(best['confidence'])}
        else:
            results = run_ultralytics_inference(lp_detector, crop, conf=0.4)
            
            if len(results) > 0 and len(results[0].boxes) > 0:
                boxes = results[0].boxes
                log(f'🔍 LP: Found {len(boxes)} plate boxes')
                debug_stats['lp_detected'] += 1
                
                # Get best
                best_conf = 0
                best_result = None
                for b in boxes:
                    conf = float(b.conf[0])
                    if conf > best_conf:
                        best_conf = conf
                        px1, py1, px2, py2 = map(int, b.xyxy[0])
                        plate_crop = crop[py1:py2, px1:px2]
                        text = read_plate_ocr(plate_crop)
                        if text != 'unknown':
                            best_result = {'text': text, 'confidence': best_conf}
                return best_result
        
        return None
    except Exception as e:
        log(f'❌ LP Error: {e}')
        import traceback; traceback.print_exc()
        return None

def check_red_light_violation(track_id, prev_y, curr_y, camera_id, img_height):
    if current_traffic_light.get(camera_id, 'UNKNOWN') != 'RED':
        return False
    line_y = (TRACK_LINE_Y / 100) * img_height
    if prev_y is not None and prev_y < line_y <= curr_y:
        return True
    return False

def check_lane_violation(vehicle_class, center_x, camera_id, img_width):
    config = camera_configs.get(camera_id)
    if not config:
        return False
    lane_points = config.get('camera_lane_track_point', [])
    lane_vehicles = config.get('camera_lane_vehicles', [])
    if not lane_points or not lane_vehicles:
        return False
    x_pct = (center_x / img_width) * 100
    boundaries = [0] + sorted(lane_points) + [100]
    for i in range(len(boundaries) - 1):
        if boundaries[i] <= x_pct < boundaries[i + 1]:
            if i < len(lane_vehicles):
                allowed = lane_vehicles[i]
                if isinstance(allowed, list):
                    if 'ANY' in [v.upper() for v in allowed]:
                        return False
                    if vehicle_class.lower() not in [v.lower() for v in allowed]:
                        return True
            break
    return False

In [ ]:
# Cell 5: Main Detection V22

def detect_frame(frame, camera_id):
    global current_traffic_light
    
    h, w = frame.shape[:2]
    result = {
        'camera_id': camera_id,
        'created_at': int(time.time() * 1000),
        'image_dimensions': {'width': w, 'height': h},
        'detections': [],
        'vehicle_count': 0,
        'tracks': [],
        'new_crossings': [],
        'license_plates': [],
        'violations': []
    }
    
    if camera_id not in camera_trackers:
        camera_trackers[camera_id] = sv.ByteTrack()
        camera_positions[camera_id] = {}
        log(f'[{camera_id[-4:]}] ByteTrack created')
    
    tracker = camera_trackers[camera_id]
    positions = camera_positions[camera_id]
    
    # Traffic Light
    if traffic_light_model:
        try:
            tl_results = run_ultralytics_inference(traffic_light_model, frame, conf=0.4)
            status = 'UNKNOWN'
            tl_dets = []
            for b in tl_results[0].boxes:
                cls_name = tl_results[0].names[int(b.cls[0])].lower()
                bx1, by1, bx2, by2 = map(float, b.xyxy[0])
                tl_dets.append({'class': cls_name, 'confidence': float(b.conf[0]), 'bbox': {'x1': bx1/w, 'y1': by1/h, 'x2': bx2/w, 'y2': by2/h}})
                if 'red' in cls_name: status = 'RED'
                elif 'green' in cls_name and status != 'RED': status = 'GREEN'
            result['traffic_light'] = {'status': status, 'detections': tl_dets}
            current_traffic_light[camera_id] = status
        except Exception as e:
            log(f'TL err: {e}')
    
    # Vehicle Detection
    if vehicle_model:
        try:
            v_results = vehicle_model(frame, classes=[2,3,5,7], conf=CONFIDENCE, verbose=False)[0]
            sv_dets = sv.Detections.from_ultralytics(v_results)
            tracked = tracker.update_with_detections(sv_dets)
            
            detections = []
            tracks_dict = {}
            detected_plates = []
            
            for i in range(len(tracked)):
                x1, y1, x2, y2 = map(int, tracked.xyxy[i])
                cx, cy = (x1+x2)//2, (y1+y2)//2
                cls_id = int(tracked.class_id[i]) if tracked.class_id is not None else 0
                cls = VEHICLE_CLASSES[min(cls_id, len(VEHICLE_CLASSES)-1)]
                conf = float(tracked.confidence[i]) if tracked.confidence is not None else 0.5
                track_id = int(tracked.tracker_id[i]) if tracked.tracker_id is not None else -1
                
                det = {
                    'class': cls,
                    'confidence': conf,
                    'bbox': {'x1': x1/w, 'y1': y1/h, 'x2': x2/w, 'y2': y2/h},
                    'center': {'x': cx, 'y': cy},
                    'track_id': track_id
                }
                tracks_dict[track_id] = {'pos': (cx, cy), 'class': cls}
                
                lp_text = None
                lp_conf = 0.0
                
                # LP Detection
                area = (x2-x1)*(y2-y1)
                if area > (w*h)*0.005:
                    lp_info = detect_license_plate(frame, x1, y1, x2, y2)
                    if lp_info and lp_info['text'] != 'unknown':
                        lp_text = lp_info['text']
                        lp_conf = lp_info['confidence']
                        det['license_plate'] = lp_text
                        detected_plates.append(lp_text)
                        if lp_text not in [p['plate'] for p in result['license_plates']]:
                            result['license_plates'].append({'plate': lp_text, 'vehicle_id': track_id, 'confidence': lp_conf})
                
                # Violations
                prev_y = positions.get(track_id)
                if check_red_light_violation(track_id, prev_y, cy, camera_id, h):
                    if not lp_text:
                        lpi = detect_license_plate(frame, x1, y1, x2, y2)
                        if lpi: lp_text = lpi['text']; lp_conf = lpi.get('confidence', 0)
                    if lp_text and lp_text != 'unknown':
                        result['violations'].append({'type': 'RED_LIGHT', 'license_plate': lp_text, 'bbox': det['bbox'], 'detection_id': track_id})
                        log(f'🚨 RED LIGHT: {lp_text}')
                
                positions[track_id] = cy
                detections.append(det)
            
            result['detections'] = detections
            result['vehicle_count'] = len(detections)
            result['tracks'] = [{'id': tid, 'class': info['class'], 'positions': [{'x': info['pos'][0], 'y': info['pos'][1], 'time': result['created_at']}]} for tid, info in tracks_dict.items()]
            
            if detected_plates:
                log(f'🚗 LP: {detected_plates}')
            
        except Exception as e:
            log(f'Vehicle err: {e}')
            import traceback; traceback.print_exc()
    
    return result

In [ ]:
# Cell 6: WebSocket
import websocket, ssl

class KaggleWebSocket:
    def __init__(self, ws_url, api_key):
        self.url = f'{ws_url}?apiKey={api_key}'
        self.ws = None
        self.connected = False
        self.send_queue = queue.Queue(maxsize=RESULT_QUEUE_SIZE)
        self._stop = False
    
    def connect(self):
        try:
            log(f'🔌 Connecting...')
            self.ws = websocket.create_connection(self.url, timeout=10, sslopt={'cert_reqs': ssl.CERT_NONE})
            self.connected = True
            log('✅ WS connected!')
            threading.Thread(target=self._sender_loop, daemon=True).start()
            return True
        except Exception as e:
            log(f'❌ WS fail: {e}')
            return False
    
    def _sender_loop(self):
        while not self._stop:
            try:
                msg = self.send_queue.get(timeout=1)
                if self.ws and self.connected:
                    self.ws.send(json.dumps(msg))
            except queue.Empty: continue
            except Exception as e: log(f'WS err: {e}'); self.connected = False
    
    def send_async(self, data):
        try: self.send_queue.put_nowait(data)
        except: pass

def fetch_cameras():
    global camera_configs, camera_keys
    try:
        resp = requests.get(f'{BACKEND_HTTP_URL}/api/camera/all', timeout=10)
        if resp.status_code == 200:
            data = resp.json()
            cams = data.get('metadata', data) if isinstance(data, dict) else data
            for cam in cams:
                if isinstance(cam, dict) and cam.get('_id'):
                    cam_id = cam['_id']
                    camera_keys[cam_id] = cam.get('camera_api_key', '')
                    camera_configs[cam_id] = {
                        'camera_lane_track_point': cam.get('camera_lane_track_point', []),
                        'camera_lane_vehicles': cam.get('camera_lane_vehicles', [])
                    }
            return list(camera_keys.keys())
    except Exception as e: log(f'Fetch err: {e}')
    return []

def test_stream(url, timeout=5):
    try:
        cap = cv2.VideoCapture(url)
        cap.set(cv2.CAP_PROP_OPEN_TIMEOUT_MSEC, timeout * 1000)
        if cap.isOpened(): ret, _ = cap.read(); cap.release(); return ret
        return False
    except: return False

In [ ]:
# Cell 7: Pipeline V22

class CameraPipelineV22:
    def __init__(self, camera_id, api_key, kaggle_ws):
        self.camera_id = camera_id
        self.flv_url = f"{NMS_URL}/live/{camera_id}.flv"
        self.frame_queue = queue.Queue(maxsize=FRAME_QUEUE_SIZE)
        self.kaggle_ws = kaggle_ws
        self._stop = False
        self.stats = {'frames_read': 0, 'frames_detected': 0, 'violations': 0, 'lp_detected': 0, 'fps': 0}
        self.fps_times = deque(maxlen=30)
        self.stream_ready = False
    
    def start(self):
        log(f'🎥 [{self.camera_id[-4:]}] Testing...')
        if not test_stream(self.flv_url, timeout=10): log(f'⚠️ [{self.camera_id[-4:]}] No stream')
        threading.Thread(target=self._reader_loop, daemon=True).start()
        threading.Thread(target=self._detector_loop, daemon=True).start()
        log(f'🚀 [{self.camera_id[-4:]}] Started')
        return True
    
    def _reader_loop(self):
        retry = 2
        while not self._stop:
            try:
                cap = cv2.VideoCapture(self.flv_url)
                cap.set(cv2.CAP_PROP_BUFFERSIZE, 1)
                if not cap.isOpened(): time.sleep(retry); retry = min(retry*2, 30); continue
                self.stream_ready = True; retry = 2
                while cap.isOpened() and not self._stop:
                    ret, frame = cap.read()
                    if not ret: break
                    self.stats['frames_read'] += 1
                    try: self.frame_queue.put_nowait(frame)
                    except: 
                        try: self.frame_queue.get_nowait(); self.frame_queue.put_nowait(frame)
                        except: pass
                cap.release(); self.stream_ready = False
            except Exception as e: log(f'Reader err: {e}'); time.sleep(retry)
    
    def _detector_loop(self):
        while not self._stop:
            try:
                frame = self.frame_queue.get(timeout=2)
                result = detect_frame(frame, self.camera_id)
                self.stats['frames_detected'] += 1
                if result.get('violations'): self.stats['violations'] += len(result['violations'])
                if result.get('license_plates'): self.stats['lp_detected'] += len(result['license_plates'])
                self.kaggle_ws.send_async(result)
                
                now = time.time()
                self.fps_times.append(now)
                if len(self.fps_times) > 1: self.stats['fps'] = round(len(self.fps_times) / (self.fps_times[-1] - self.fps_times[0]), 1)
                
                if self.stats['frames_detected'] % 50 == 0:
                    tl = result.get('traffic_light', {}).get('status', '-')
                    log(f"[{self.camera_id[-4:]}] {self.stats['fps']}FPS | Det:{len(result.get('detections',[]))} | TL:{tl} | LP:{self.stats['lp_detected']} | Viol:{self.stats['violations']}")
                    log(f"📊 Debug: {debug_stats}")
            except queue.Empty: continue
            except Exception as e: log(f'Det err: {e}')

In [ ]:
# Cell 8: MAIN
log('🔍 Fetching cameras...')
cameras = fetch_cameras()

if not cameras:
    log('❌ No cameras!')
else:
    log(f'✅ Found {len(cameras)} cameras')
    pipelines = []
    for cam_id in cameras:
        ws = KaggleWebSocket(BACKEND_WS_URL, camera_keys.get(cam_id, ''))
        if not ws.connect(): continue
        p = CameraPipelineV22(cam_id, camera_keys.get(cam_id, ''), ws)
        if p.start(): pipelines.append(p)
        time.sleep(0.5)
    
    log(f'🚀 {len(pipelines)} PIPELINES!')
    try:
        while True:
            time.sleep(60)
            for p in pipelines:
                log(f"📊 [{p.camera_id[-4:]}] R:{p.stats['frames_read']} D:{p.stats['frames_detected']} FPS:{p.stats['fps']} LP:{p.stats['lp_detected']} V:{p.stats['violations']}")
    except KeyboardInterrupt:
        log('Stop')